In [1]:
import pennylane as qml
import autograd.numpy as np_grad
import numpy as np

https://quantumcomputing.stackexchange.com/questions/17637/how-to-create-the-equivalent-of-the-qiskit-rxx-gate-in-pennylane

https://pennylane.readthedocs.io/en/stable/code/api/pennylane.MultiRZ.html

In [15]:
n_qubit = 10

In [16]:
# def RXX(theta):
#     rxx = np_grad.array([
#         [np_grad.cos(theta/2), 0, 0, - 1j*np_grad.sin(theta/2)],
#         [0, np_grad.cos(theta/2), - 1j*np_grad.sin(theta/2), 0],
#         [0, - 1j*np_grad.sin(theta/2), np_grad.cos(theta/2), 0],
#         [- 1j*np_grad.sin(theta/2), 0, 0, np_grad.cos(theta/2)]
#     ])
#     return rxx

# def RZZ(theta):
#     rzz = np.array([
#         [np_grad.cos(theta/2) - 1j*np_grad.sin(theta/2), 0, 0, 0],
#         [0, np_grad.cos(theta/2) + 1j*np_grad.sin(theta/2), 0, 0],
#         [0, 0, np_grad.cos(theta/2) + 1j*np_grad.sin(theta/2), 0],
#         [0, 0, 0, np_grad.cos(theta/2) - 1j*np_grad.sin(theta/2)]
#     ])
#     return rzz

# dev = qml.device('default.qubit', wires=2)
# @qml.qnode(dev)
# def circuit(theta):
#     qml.QubitUnitary(RXX(theta), wires=[0, 1])
#     return qml.expval(qml.PauliZ(0))

In [17]:
from pennylane.operation import Operation

class RXX(Operation):
    num_params = 1
    num_wires = 2
    par_domain = "R"

    grad_method = "A"
    grad_recipe = None # This is the default but we write it down explicitly here.

    generator = [(qml.PauliX(0) @ qml.PauliX(1)).matrix, -0.5]

    @staticmethod
    def decomposition(theta, wires):
        return [qml.PauliRot(theta, 'XX', wires=wires)]

    @staticmethod
    def _matrix(*params):
        theta = params[0]
        c = np.cos(0.5 * theta)
        s = np.sin(0.5 * theta)
        return np.array(
            [
                [c, 0, 0, -s],
                [0, c, -s, 0],
                [0, -s, c, 0],
                [-s, 0, 0, c]
            ]
        )

    def adjoint(self):
        return RXX(-self.data[0], wires=self.wires)

In [18]:
def squeezing(params):
    
    # RY gate
    for i in range(n_qubit):
        qml.RY(np_grad.pi/2, wires=i)
    
    # GMS_z gate
    for i in range(n_qubit):
        for j in range(i+1, n_qubit):
            qml.MultiRZ(params[0][i][j], wires=[i, j])
    
    # RX gate
    for i in range(n_qubit):
        qml.RX(params[1][0][i], wires=i)
    
    # GMS_x gate
    for i in range(n_qubit):
        for j in range(i+1, n_qubit):
            RXX(params[2][i][j], wires=[i, j])

In [19]:
dev = qml.device('default.qubit', wires=n_qubit)

@qml.qnode(dev)
def squeezing_circuit_X(params):
    
    squeezing(params)
#     qml.adjoint(squeezing)(params)
    
    # Measurement 
    return [qml.expval(qml.PauliX(i)) for i in range(n_qubit)]

@qml.qnode(dev)
def squeezing_circuit_Z(params):
    
    squeezing(params)
#     qml.adjoint(squeezing)(params)
    
    # Measurement 
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubit)]

In [20]:
def cost(params):
    
    expval_X = squeezing_circuit_X(params)
    expval_Z = squeezing_circuit_Z(params)
    
    return len(expval_X)*np_grad.sum(expval_Z**2)/np_grad.sum(expval_X**2)

In [21]:
params = np_grad.random.normal(size=(3, n_qubit, n_qubit))

In [23]:
cost(params)

tensor(2.47110123e-28, requires_grad=True)

In [ ]:
opt = qml.AdamOptimizer()
costs_array = [cost(params)]
steps = 100
for i in range(steps):
    params = opt.step(cost, params)
    costs_array.append(cost(params))

In [ ]:
costs_array[-10:]